In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pickle
import gzip
import time
from warnings import filterwarnings
filterwarnings('ignore')
from _utils import utils, transforming, filling

data = ['../BigContest_data/' + file for file in os.listdir('../BigContest_data')]

# 고객 특성에 대한 feature 추출
(use : user_spec)

A 변수의 범주별 B 변수의 대표값 추출<br>

B가 범주형일 때는 각 범주별 차지하는 비율<br>
B가 연속형일 때는 max,min,average,std,1분위수,2분위수(median),3분위수을 추출한다.

A가 연속형일 때는 구간화하여 범주형으로 만든다.

extracted feature name : {B} by {A}={대표값}

In [2]:
# 분석 전과정은 train set으로 한다.
# 단 구간화와 관련된 부분은 data leakage의 위험이 없으므로 한꺼번에 처리한다.
user_spec_tr_te = pd.read_csv('../preprocessed/user_spec_filled.csv')

In [3]:
A_cat = ['houseown_type','gender','income_type','employment_type','purpose','personal_rehabilitation_yn','personal_rehabilitation_complete_yn' ]

A_conti = ['birth_year','credit_score','yearly_income','company_enter_month']

B_cat = ['purpose']
B_conti = ['yearly_income','desired_amount','existing_loan_amt']

## 연속형 변수의 구간화(위의 A_conti 변수들)
- 구간화 변수 만드는 train만 따로 구분짓지 않음 
- (구간을 직접 지정하므로 data leakage에 대한 우려 없음)


### credit score
- 100단위로 구간화


In [ ]:
user_spec_tr_te['credit_score'].min(), user_spec_tr_te['credit_score'].max()

In [ ]:
credit_score_cut = pd.cut(user_spec_tr_te['credit_score'], range(0,1100,100), labels=np.arange(0,1000,100)+1)
credit_score_cut = credit_score_cut.astype(object)
credit_score_cut.value_counts().sort_index()

### yearly income
- 소득세율 기준에 따른 구간화

In [ ]:
user_spec_tr_te['yearly_income'].min(), utils.split_comma(user_spec_tr_te['yearly_income'].max())

In [ ]:
yearly_income_cut = pd.cut(user_spec_tr_te['yearly_income'], [-1,14000000, 50000000, 88000000, 150000000, 300000000, 500000000, 1000000000, np.inf],
                          labels=['grade1','grade2','grade3','grade4','grade5','grade6','grade7','grade8'])

yearly_income_cut = yearly_income_cut.astype(object)
temp = yearly_income_cut.value_counts().to_frame()
temp['order'] = yearly_income_cut.value_counts().index.str[-1].astype(int)
temp.sort_values('order').drop('order',axis=1).rename(columns={'yearly_income':'count'})

### 입사년차
- 년 단위로

In [8]:
as_int = lambda x:str(x).replace('.0','') if pd.isna(x)==False else x
user_spec_tr_te['company_enter_month'] = user_spec_tr_te['company_enter_month'].map(as_int)

In [ ]:
enter_period_year = pd.to_datetime(user_spec_tr_te['insert_time']).dt.year - pd.to_datetime(user_spec_tr_te['company_enter_month']).dt.year
enter_period_year *= 12
enter_period_month = pd.to_datetime(user_spec_tr_te['insert_time']).dt.month - pd.to_datetime(user_spec_tr_te['company_enter_month']).dt.month
enter_period = enter_period_year+enter_period_month
enter_period /= 12

print(enter_period.min(), enter_period.max())

In [ ]:
# 명백한 입력 오류가 존재하니 출생보다 입사일시가 빠른 사람은 null 값 처리
user_spec_tr_te[enter_period>100].head()

company_enter_month가 null인 사람의 경우 단순 누락인지, 주부 혹은 프리랜서인지 경우가 다를 수 있다.<br>
다음과 같은 처리를 해주도록 한다.
1. 15살 이전 입사 : 누락값 취급
    - birth가 없는 사람은 계산 불가이므로 2022년을 채워넣고 입사년수가 모두 0 이하가 나오게 한다.
2. 입사년도가 NaN일때
    - case1) yearly_income =0 -> 미취업
    - case2) case1이 아니면서, employment_type이 기타 -> 비정기소득자
    - case3) case1이 아니면서, employment_type이 기타도 아님 -> 안채워넣음


In [ ]:
# 1. 15살 이전 입사는 누락값
enter_age = pd.to_datetime(user_spec_tr_te['company_enter_month']).dt.year - user_spec_tr_te['birth_year'].fillna(2022) + 1
enter_period[enter_age <= 15] = np.nan
print(enter_period.min(), enter_period.max())

In [12]:
# 구간화
enter_period_cut = pd.cut(enter_period, [-np.inf]+list(range(1,54,2)), labels=range(-1,52,2)).replace({-1:0})

In [13]:
# 2. 입사년도가 NaN일 때
def fill_enter_year(user_spec):
    if user_spec['yearly_income'] == 0:
        y = 'no_job'
    elif user_spec['employment_type']=='etc':
        y = 'no_regular_income'
    else:
        y = np.nan
    return y
enter_period_cut_null = user_spec_tr_te[user_spec_tr_te['company_enter_month'].isnull()].apply(fill_enter_year, axis=1)

In [14]:
enter_period_cut[user_spec_tr_te['company_enter_month'].isnull()] = enter_period_cut_null

In [ ]:
enter_period_cut.isnull().sum()

### 나이
- 5단위로 끊되 81세 이상은 하나로 합치기로 한다.

In [ ]:
age = pd.to_datetime(user_spec_tr_te['insert_time']).dt.year - user_spec_tr_te['birth_year'] + 1

age.min(), age.max()

In [ ]:
age_cut = pd.cut(age,range(15,100,5), labels=np.arange(15,95,5)+1)
age_cut = age_cut.map(lambda x:81 if x>=80 else x)
age_cut.value_counts().sort_index()

## user_spec에 새 컬럼 추가 및 수정

In [19]:
# age
user_spec_tr_te['age'] = age
# 입사년차
user_spec_tr_te['company_period'] = enter_period

user_spec_tr_te['enter_period_cut'] = enter_period_cut
user_spec_tr_te['yearly_income_cut'] = yearly_income_cut
user_spec_tr_te['credit_score_cut'] = credit_score_cut
user_spec_tr_te['age_cut'] = age_cut

## 각 속성별 대표값 추출하기
- 대표값은 전체를 다 계산해 내는 통계이므로 data leakage를 막기 위해 train set만 사용하기로 한다.

In [20]:
user_spec_train, user_spec_test = utils.split_train_test(user_spec_tr_te,'insert_time')

In [21]:
utils.save_pickle(user_spec_test, '../preprocessed/user_spec_test_bin.pickle')

The Object saved in ../preprocessed/user_spec_test_bin.pickle.


### 각 특성별 대표값 사전
- test set에 대해서는 이 사전을 이용해 매핑한다.

In [20]:
A_cat = ['houseown_type','gender','income_type','employment_type','purpose','personal_rehabilitation_yn','personal_rehabilitation_complete_yn' ]
A_conti = ['age_cut','credit_score_cut','yearly_income_cut','enter_period_cut']

B_conti = ['yearly_income','desired_amount','existing_loan_amt']
B_cat = ['purpose']


In [21]:
start = time.time()

mapping_dict = transforming.get_mapping_dict(user_spec_train, 
                                             A_cat+A_conti,
                                             B_conti, B_cat)

print((time.time()-start)/60,'분')

0.12696184317270914 분


### 변환

In [ ]:
# 예시(연속형)
transforming.table_transform_conti(user_spec_train,'houseown_type','desired_amount').head()

In [ ]:
# 비교
display(user_spec_train[['houseown_type']].head())
print(mapping_dict[('houseown_type','desired_amount')])

In [ ]:
# 예시(범주형)
transforming.table_transform_cat(user_spec_train,'houseown_type','purpose').head()

In [ ]:
# 비교
display(user_spec_train[['houseown_type']].head())
print(mapping_dict[('houseown_type','purpose')])

In [22]:
user_spec_cols = pd.read_csv('../preprocessed/user_spec_filled.csv').columns.tolist()
# 'birth_year','company_enter_month'는 이미 age와 company_period로 들어갔으니 제거
all_info = user_spec_train.loc[:,user_spec_cols + ['age','company_period']].drop(['birth_year','company_enter_month'], axis=1)

In [ ]:
from itertools import product
for A,B in product(A_cat+A_conti, B_conti):
    if B in A:
        continue
    try:
        all_info = pd.concat([all_info,transforming.table_transform_conti(user_spec_train,A,B)],axis=1)
        print(f'{A} and {B} completed')
    except:
        print(f'check {A} and {B}')

In [ ]:
for A,B in product(A_cat+A_conti, B_cat):
    if B in A:
        continue
    try:
        all_info = pd.concat([all_info,transforming.table_transform_cat(user_spec_train,A,B)],axis=1)
        print(f'{A} and {B} completed')
    except:
        print(f'check {A} and {B}')

In [ ]:
all_info.shape

In [85]:
utils.save_pickle(mapping_dict, '../preprocessed/mapping_dict.pickle')
# 저장되는데 시간이 오래걸림
utils.save_pickle(all_info, '../preprocessed/all_info.pickle')

The Object saved in ../preprocessed/mapping_dict.pickle.
The Object saved in ../preprocessed/all_info.pickle.


In [9]:
utils.save_pickle(all_info, '../preprocessed/all_info.pickle')

The Object saved in ../preprocessed/all_info.pickle.


### user_spec과 추출된 변수로부터 의미있는 변수를 선정하기 위한 임시 종속변수

In [ ]:
applied_application_id = loan_result_train.query('is_applied==1')['application_id'].unique()
all_info_is_applied = user_spec_train['application_id'].map(lambda x:1 if x in applied_application_id else np.nan)

In [29]:
utils.save_pickle(all_info_is_applied, '../preprocessed/temp_is_applied.pickle')

The Object saved in ../preprocessed/temp_is_applied.pickle.


# 상품 특성에 대한 feature 추출
(use :  loan_result)

- application 건별 loan_limit 금액 합
- application 건별 loan_rate의 대표값(평균, 중앙값,...) 
- 은행(bank_id)별 limit, rate의 대표값(평균, 중앙값,...)
- 상품(product_id)별 limit, rate의 대표값(평균, 중앙값,...)
- 10분위 desired 금액 구간 대비 limit, rate 비율 
- desired amount 대비 loan_limit 비율

In [28]:
# 결측치가 제거되지 않은 데이터를 이용해 더 많은 feature 정보를 담아내기로 한다.
# 제거해야할 결측치는 모델링과정에서 모두 제거된다.
loan_result = pd.read_csv(data[2])
loan_result_train, loan_result_test = utils.split_train_test(loan_result,'loanapply_insert_time')

In [13]:
del loan_result

In [16]:
loan_result_train = pd.merge(loan_result_train,user_spec_tr_te[['application_id','desired_amount']], on='application_id', how='left' )

In [ ]:
# qcut을 사용했으므로 train만으로 구간화
desired_cut_train = pd.qcut(loan_result_train['desired_amount'],10,labels = [f'grade_{i}' for i in range(1,11)])

desired_cut_train = desired_cut_train.astype(str)
desired_cut_train.head()

In [18]:
loan_result_train['desired_amount_cut'] = desired_cut_train

In [27]:
# 0으로 나눠지는 일을 방지 -> nan값으로 처리
loan_result_train2 = loan_result_train.copy()
loan_result_train2.loc[loan_result_train2['desired_amount'] == 0,'desired_amount'] = np.nan
loan_result_train['loan_limit_per_desired_amount'] = loan_result_train['loan_limit']/loan_result_train2['desired_amount']
del loan_result_train2

In [ ]:
# 얘를 어떻게 바꿀까?
# 주의 : application_id로 grouping하는 마지막 연산은 상당히 오래걸릴 수 있음
loan_info = loan_result_train.copy()

AB = [('bank_id','loan_limit'),('bank_id','loan_rate'),('product_id','loan_limit'),('product_id','loan_rate'),
     ('desired_amount_cut','loan_limit'),('desired_amount_cut','loan_rate'),('application_id','loan_rate')]
conti_cat = ['conti','conti','conti','conti','conti','conti','conti']
for i,(A,B) in enumerate(AB):
    try:
        loan_info = pd.concat([loan_info,transforming.table_transform_conti(loan_result_train, A,B)],axis=1)
        print(f'{A} and {B} completed')
    except:
        print(f'check {A} and {B}')
        

In [34]:
utils.save_pickle(loan_info, '../preprocessed/loan_info.pickle')

The Object saved in ../preprocessed/loan_info.pickle.


## 사전 매핑

In [149]:
AB = [('bank_id','loan_limit'),('bank_id','loan_rate'),('product_id','loan_limit'),('product_id','loan_rate'),
     ('desired_amount_cut','loan_limit'),('desired_amount_cut','loan_rate')]

mapping_dict_loan = transforming.get_mapping_dict(loan_result_train,A=list(np.array(AB)[:,0]), B_conti=list(np.array(AB)[:,1]),B_cat=[] )

In [150]:
utils.save_pickle(mapping_dict_loan, '../preprocessed/mapping_dict_loan.pickle')

The Object saved in ../preprocessed/mapping_dict_loan.pickle.
